<a href="https://colab.research.google.com/github/FuturisticLover123/learning/blob/main/%D0%A2%D1%80%D0%B5%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B8%20%D0%BF%D0%BE%20ML%203.0/task6/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

image preprocessing

In [ ]:
!mkdir -p images


Archive:  dataset.zip
caution: filename not matched:  -C
caution: filename not matched:  /content/images


In [ ]:
!unzip dataset.zip -d /content/images

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: /content/images/dataset/1983.png  
  inflating: /content/images/dataset/2919.png  
  inflating: /content/images/dataset/4957.png  
  inflating: /content/images/dataset/0755.png  
  inflating: /content/images/dataset/7764.png  
  inflating: /content/images/dataset/1886.png  
  inflating: /content/images/dataset/5117.png  
  inflating: /content/images/dataset/1544.png  
  inflating: /content/images/dataset/7700.png  
  inflating: /content/images/dataset/7237.png  
  inflating: /content/images/dataset/3554.png  
  inflating: /content/images/dataset/2639.png  
  inflating: /content/images/dataset/4796.png  
  inflating: /content/images/dataset/5412.png  
  inflating: /content/images/dataset/5752.png  
  inflating: /content/images/dataset/6134.png  
  inflating: /content/images/dataset/4565.png  
  inflating: /content/images/dataset/0960.png  
  inflating: /content/images/dataset/7379.png  
  inflating: /conten

In [105]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os

In [106]:
model = models.resnet101(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])
model.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [107]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_embedding(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)

    with torch.no_grad():
        embedding = model(img_tensor)

    return embedding.squeeze().numpy()


In [108]:
image_path = "/content/images/dataset/0000.png"
embedding = get_embedding(image_path)
print(f"Размер эмбеддинга: {embedding.shape}")

Размер эмбеддинга: (2048,)


In [111]:
import numpy as np
from tqdm import tqdm
num_images = 9605
embedding_size = 2048
embeddings_array = np.zeros((num_images, embedding_size), dtype=np.float32)

dir_path = "/content/images/dataset"
for i in tqdm(range(9605)):
  image_path = f"{dir_path}/{str(i).zfill(4)}.png"
  embeddings_array[i] = get_embedding(image_path)

100%|██████████| 9605/9605 [1:06:07<00:00,  2.42it/s]


In [125]:
np.save("embadings_restnet101.npy",embeddings_array)

In [119]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 60.4 MB/s eta 0:00:00


In [122]:
import faiss
index = faiss.IndexFlatIP(2048)
index.add(embeddings_array)

In [161]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_nearest_embeddings(query_embedding, embeddings_matrix, top_k=6):

    similarities = cosine_similarity([query_embedding], embeddings_matrix)[0]

    top_indices = np.argpartition(-similarities, top_k)[:top_k]

    sorted_indices = top_indices[np.argsort(-similarities[top_indices])]

    return sorted_indices[1:], similarities[sorted_indices][1:]

In [173]:
matrix = np.empty((9605, 2), dtype='U100')

In [174]:
for i, embeding in tqdm(enumerate(embeddings_array)):
  matrix[i, 0] = f"{str(i).zfill(4)}.png"
  indexes, distance = find_nearest_embeddings(embeding, embeddings_array, 7)
  matrix[i, 1] = " ".join(map(lambda i: f"{str(i).zfill(4)}.png", indexes))

9605it [17:43,  9.03it/s]


In [182]:
import pandas as pd
pd.DataFrame(matrix).to_csv("submission.csv", index=False, header = ["filename", "ranking"])

In [179]:
pd.DataFrame(matrix)

,0,1
0,0000.png,8222.png 6448.png 4932.png 8492.png 9405.png 2...
1,0001.png,5029.png 6470.png 5193.png 1149.png 3986.png 3...
2,0002.png,7688.png 3977.png 5492.png 2448.png 1858.png 5...
3,0003.png,6326.png 6438.png 9169.png 5911.png 3675.png 5...
4,0004.png,4593.png 4191.png 6371.png 7238.png 5676.png 1...
...,...,...
9600,9600.png,7663.png 0268.png 9135.png 0896.png 7670.png 2...
9601,9601.png,8581.png 0213.png 3844.png 4463.png 2344.png 0...
9602,9602.png,7691.png 2158.png 5991.png 4789.png 9189.png 6...
9603,9603.png,6298.png 0893.png 6401.png 8022.png 8816.png 1...
